In [0]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [8]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [9]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [10]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.SimpleRNN(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.SimpleRNN(64,return_sequences=True,dropout=0.2),
    tf.keras.layers.SimpleRNN(32,return_sequences=True,dropout=0.1),
    tf.keras.layers.SimpleRNN(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 200, 128)          20608     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 200, 64)           12352     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 200, 32)           3104      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,037,016
Trainable params: 8,037,016
Non-trainable params: 0
______________________________________________

In [11]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 287s 890ms/step - loss: 2.0720 - acc: 0.1501 - val_loss: 2.0583 - val_acc: 0.1603
Epoch 2/30
323/323 [==============================] - 287s 889ms/step - loss: 2.0543 - acc: 0.1653 - val_loss: 2.0590 - val_acc: 0.1603
Epoch 3/30
323/323 [==============================] - 287s 889ms/step - loss: 2.0361 - acc: 0.1739 - val_loss: 1.9917 - val_acc: 0.2049
Epoch 4/30
323/323 [==============================] - 286s 885ms/step - loss: 1.9277 - acc: 0.2294 - val_loss: 1.9124 - val_acc: 0.2261
Epoch 5/30
323/323 [==============================] - 281s 871ms/step - loss: 1.8567 - acc: 0.2553 - val_loss: 1.9219 - val_acc: 0.2219
Epoch 6/30
323/323 [==============================] - 282s 872ms/step - loss: 1.7998 - acc: 0.2906 - val_loss: 1.8304 - val_acc: 0.2643
Epoch 7/30
323/323 [==============================] - 280s 867ms/step - loss: 1.7322 - acc: 0.3102 - val_loss: 1.8049 - val_acc: 0.2882
Epoch 8/30
323/323 [============================

In [12]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 6400)
(75271, 8)
(16524, 1, 6400)
(16524, 8)


In [0]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.SimpleRNN(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.SimpleRNN(1024,return_sequences=True,dropout=0.4,recurrent_dropout=0.3),
    tf.keras.layers.SimpleRNN(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.2),
    tf.keras.layers.SimpleRNN(256,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.SimpleRNN(64,return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [16]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 1, 2048)           17303552  
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 1, 1024)           3146752   
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 1, 512)            786944    
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 1, 256)            196864    
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 1, 64)             20544     
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                

In [17]:
history = model.fit(X_train, y_train, epochs=25,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/25
265/265 [==============================] - 6s 24ms/step - loss: 1.4224 - acc: 0.4255 - val_loss: 1.2389 - val_acc: 0.5012
Epoch 2/25
265/265 [==============================] - 6s 21ms/step - loss: 1.2583 - acc: 0.4982 - val_loss: 1.1724 - val_acc: 0.5392
Epoch 3/25
265/265 [==============================] - 6s 22ms/step - loss: 1.2135 - acc: 0.5206 - val_loss: 1.1426 - val_acc: 0.5473
Epoch 4/25
265/265 [==============================] - 6s 21ms/step - loss: 1.1888 - acc: 0.5289 - val_loss: 1.1206 - val_acc: 0.5607
Epoch 5/25
265/265 [==============================] - 6s 21ms/step - loss: 1.1722 - acc: 0.5349 - val_loss: 1.1033 - val_acc: 0.5684
Epoch 6/25
265/265 [==============================] - 6s 21ms/step - loss: 1.1577 - acc: 0.5411 - val_loss: 1.0959 - val_acc: 0.5739
Epoch 7/25
265/265 [==============================] - 6s 21ms/step - loss: 1.1494 - acc: 0.5458 - val_loss: 1.0907 - val_acc: 0.5643
Epoch 8/25
265/265 [==============================] - 6s 22ms/step - 

In [0]:
!ls saved_model

text_classifier_rnn_word_embedding_epoch_15
text_classifier_rnn_word_embedding_epoch_20


In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [19]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 2s 4ms/step - loss: 1.0590 - acc: 0.5812
[1.05898916721344, 0.5812152028083801]


In [0]:
y_pred=model.predict(X_test)

In [20]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [21]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])

print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

59.32246823575644
55.32136800010691
55.57173747274706
